In [1]:
import argparse
import os
import numpy as np
import tensorflow.keras
from PIL import Image
import h5py
import glob
import random
import cv2
import csv
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
#getting the directory of dataset
IMG_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\images"
DATA_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\data"
NEWDATA_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\resized_data"
img_row, img_cols = 200, 200

#keep track of deleted images
del_imgs = set()
for file in os.listdir(IMG_DIR):
    im = Image.open(IMG_DIR+'\\'+file)
    if(im.size >= (200, 200)):  
        #resize images
        img = im.resize((img_row, img_cols))
        #convert to grayscale
        gray = im.convert('L')
        gray.save(NEWDATA_DIR+'\\'+file,"JPEG")
    else:
        del_imgs.add(file)
        continue

#images = os.listdir(NEWDATA_DIR)
#tester = array(Image.open('resized_data'+'\\'+image))

In [98]:
test = r"./dsSwampers.hdf5"

In [99]:
IMG_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\images"
DATA_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\data"
NEWDATA_DIR = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\resized_data"
def read_img(NEWDATA_DIR):
    fileList = []

    for rot, dirs, files in os.walk(NEWDATA_DIR):
        for file in files:
            if(file.endswith("jpg") or file.endswith("JPG")):
                temp = str(NEWDATA_DIR+file)
                fileList.append(temp)
    return fileList

In [100]:
thisDict = dict()
files = os.listdir(DATA_DIR)
with open(DATA_DIR+'\\'+'500_picts_satz.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    with open(DATA_DIR+'\\'+'new_data.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        thisDict = {IMG_DIR+'\\'+row[1]:row[2] for row in csv_reader if row[2] == 'happiness' or row[2] == 'neutral'}

In [101]:
happiness = []
boredom = []
for key in thisDict:
    if(thisDict[key] == 'happiness'):
        happiness.append(key)
    else:
        boredom.append(key)
hapAlt = random.sample(happiness, len(boredom))
boreAlt = random.sample(boredom, len(boredom))

label_bumps = np.ones(shape=(len(hapAlt),), dtype=np.uint8).tolist()
label_trash = np.zeros(shape=(len(boreAlt),), dtype = np.uint8).tolist()

img_addr = hapAlt
img_label = label_bumps + label_trash

In [102]:
matches = list(zip(img_addr, img_label))
random.shuffle(matches)
(addrs, labels) = zip(*matches)

In [103]:
train_addr = list(addrs[0:int(len(addrs))])
train_labels = list(labels[0:int(len(labels))])
print(len(train_addr), len(train_labels))
img_shape = (len(img_addr),100, 100, 3)

84 84


In [104]:
f = h5py.File(test, mode="w")
f.create_dataset("train_img", img_shape, np.float32)
f.create_dataset("train_label", (len(train_labels), ), np.float32)
f["train_label"][...] = train_labels
x,y = f.values()
print(x)
print(y)

<HDF5 dataset "train_img": shape (84, 100, 100, 3), type "<f4">
<HDF5 dataset "train_label": shape (84,), type "<f4">


In [105]:
bad_indexes = []
for i in range(len(img_addr)):
  
    addr_read = img_addr[i]
    img = cv2.imread(addr_read)
    img = cv2.resize(img, (100,100), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    f["train_img"][i, ...] = img[None]

    #resize images
    #img = img.resize((img_row, img_cols))
    #convert to grayscale
    #gray = img.convert('L')
    #gray.save(NEWDATA_DIR+'\\'+file,"JPEG")
print(bad_indexes)

[]


In [106]:
import pickle
from sklearn.externals import joblib 
def CNN_training(x_train, y_train):
    
        
    x_train = np.array(x_train)
    x_train = x_train/255.0 #normalize
    
    
    y_train = np.array(y_train)
    
    
    #start
    model = Sequential()
    #insert a conv layer that accepts the image data x2
    
    ### READ THIS ####
    #You had an error in the input shape, where you se (50, 50 ,1) and it should be (50, 50, 3)
    # remember you need to be consisted with the size
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100,3)))
    
    #max pool layer for further processing x2
    model.add(keras.layers.MaxPooling2D((2,2)))
    model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    
    #flatten data for full-connected layer
    model.add(keras.layers.Flatten())
    
    #regularization
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(512,activation='relu'))
    model.add(keras.layers.Dense(2,activation='sigmoid'))
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    #ntrain = len(x_train)
    #nval = len(x_test)
    
    
    model.fit(x_train,y_train,epochs=1,batch_size=32)

    model.save('CNN_testerSwamp.sav')

In [107]:

test = r"./dsSwampers.hdf5"
# read the hdf5 or h5 time, use mode "r" for "Only Read"
f = h5py.File(test, mode="r")
x, y = f.values()
print(x.shape)
print(y.shape)
y_binary = to_categorical(y)
print(y_binary)
print(x)
CNN_training(x,y_binary)

(84, 100, 100, 3)
(84,)
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
<HDF5 dataset "train_img": shape (84, 100, 100, 3), type "<f4">
84/84 [==============================] - 2s 23ms/sample - loss: 0.2686 - acc: 0.7321


In [109]:
img2 = keras.preprocessing.image.load_img('C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\images\\Sourav_Ganguly_0004.jpg') # this is a PIL image
print(img2)
array2 = keras.preprocessing.image.img_to_array(img)    
arrayresized2 = cv2.resize(array2, (100,100))
print(arrayresized2) # this is a Numpy array with shape (3, 150, 150)
inputarray2 = arrayresized[np.newaxis,...] # dimension added to fit input size
img2 = keras.preprocessing.image.load_img('C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\images\\Spencer_Abraham_0005.jpg') # this is a PIL image
print(img2)
array2 = keras.preprocessing.image.img_to_array(img2)    
arrayresized2 = cv2.resize(array, (100,100))
print(arrayresized2) # this is a Numpy array with shape (3, 150, 150)
inputarray = arrayresized[np.newaxis,...] # dimension added to fit input size
net = tf.keras.models.load_model(r'C:\Users\JohnQ\Documents\Hackathons\SwampHacks2020\CNN_testerSwamp.sav')

print(net.predict_classes(inputarray2))
print(net.predict_classes(inputarray))

<PIL.Image.Image image mode=RGB size=350x350 at 0x2293BE342B0>
[[[ 57.  57.  57.]
  [ 57.  57.  57.]
  [ 58.  58.  58.]
  ...
  [ 43.  43.  43.]
  [ 48.  48.  48.]
  [ 43.  43.  43.]]

 [[ 57.  57.  57.]
  [ 56.  56.  56.]
  [ 58.  58.  58.]
  ...
  [ 42.  42.  42.]
  [ 48.  48.  48.]
  [ 47.  47.  47.]]

 [[ 57.  57.  57.]
  [ 56.  56.  56.]
  [ 57.  57.  57.]
  ...
  [ 43.  43.  43.]
  [ 45.  45.  45.]
  [ 47.  47.  47.]]

 ...

 [[ 59.  59.  59.]
  [ 59.  59.  59.]
  [ 61.  61.  61.]
  ...
  [ 93.  93.  93.]
  [ 97.  97.  97.]
  [ 92.  92.  92.]]

 [[ 53.  53.  53.]
  [ 52.  52.  52.]
  [ 53.  53.  53.]
  ...
  [ 97.  97.  97.]
  [ 99.  99.  99.]
  [ 94.  94.  94.]]

 [[ 48.  48.  48.]
  [ 48.  48.  48.]
  [ 46.  46.  46.]
  ...
  [ 99.  99.  99.]
  [102. 102. 102.]
  [ 95.  95.  95.]]]
<PIL.Image.Image image mode=RGB size=350x350 at 0x228DE555668>
[[[ 44.5625  44.5625  44.5625]
  [  4.1875   4.1875   4.1875]
  [  6.25     6.25     6.25  ]
  ...
  [ 14.25    14.25    14.25  ]
  [ 14

In [114]:
import pickle
from sklearn.externals import joblib
from keras.models import load_model
cnn = keras.models.load_model('CNN_tester.model')
#cnn = "C:\\Users\\JohnQ\\Documents\\Hackathons\\SwampHacks2020\\CNN_tester2.sav"
saved_model = pickle.dumps(cnn)
cnn_from_pickle = pickle.loads(saved_model) 
print(cnn_from_pickle)  
# Save the model as a pickle in a file 
joblib.dump(cnn, 'modelSwampers.pkl') 
  
# Load the model from the file 
cnn_from_joblib = joblib.load('modelSwampers.pkl')


TypeError: can't pickle _thread.RLock objects